*Please note, this directory is independent of the edward package such that requirements must be downloaded seperately and are not provided in `requirements.txt`.

# Plink v. Edward: PCA

## Time
Run `plink_edward.sh` with input file of choice

## Accuracy
1. Compare html output of Edward to code below to read in Plink output